<a href="https://colab.research.google.com/github/drdww/OPIM5641/blob/main/Module4/M4_1/2_Introduction_and_Activation_Variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integer Programming: Introduction and Activation Variables

* `Powell`: Chapter 11 (Integer Optimization)


# Integer Programming

We saw several examples where the optimal solutions (i.e., the values of the decision variables delivering the best objective value) were fractional numbers. However, in many practical examples, we would like these values to be integer.

## Sometimes, Linear Programming gives us integer solutions

In a few cases (very few, unfortunately), optimal LP solutions have integer values. For instance, we saw this happening when we solve certain types of  network problems (e.g., the shortest path problem). More generally, given a linear model with constraints $Ax \leq b$, if 
$b$ contains only integer values and if $A$ is [totally unimodular](https://en.wikipedia.org/wiki/Unimodular_matrix), then we know that the an optimal solution for this value will have all decision variables assigned to integer values (this is advanced material, you don't need to know this for this course).

In most cases (virtually all interesting examples in practice, unfortunately), linear programs will give fractionary solutions. Therefore, we need to use **integer programming**, which is about models that force (some of the) variables to be integers.

## Why we just don't round when we get fractional solutions?

In practice, one will typically take a fractional solution and round the values so as to get something that makes physical sense. In practice, rounding is useful, but it can be problematic because:
* It can lead to infeasible solutions
* In some cases, there is no obvious rounding (e.g., choice variables are all equally distributed among the possible alternatives)
* Rounded solutions may have very bad quality (i.e., they may be much worse than the actual optimal solution)

## Why is Integer Programming nice?

You can express **way more** problems using IP than with LP. Also, IPs occur **much more** frequently than LPs:
* Scheduling (workers, planes, machines, trucks, assignment of classes to rooms at UConn :-) )
* Manufacturing
* Transportation (e.g., UPS)
* Telecommunications (e.g., AT&T)
* Advertising (e.g., MSNBC)
IP is so important because in most real-world problems either **variables need to be integers**, or you need to satisfy a number of **logical constraints**, which may be formulated using **binary variables**, i.e., variables that can only be equal to 0 or to 1.

## Is there any problem with Integer Programming?

Yes, unfortunatelly, IPs are **much harder** to solve than LPs. In practice, some LP problems with millions of variables may be solved in minutes or seconds. On the other hand, certain IP problems with only 100 variables can take **years** (!!) to solve.

## How do we solve Integer Programs?

For this course, all you need to know is that Pyomo can use some solvers that can handle Integer Programs. Behind the curtains, these solvers check all possible assignments of integer values to variables (i.e., they perform a brute-force search). In some cases, they can infer that some combinations of values are not promising and do not need to be examined explicitly; coming up with the right inferences is the art behind these solvers.

This procedure may require the solution of a large number of linear programs. This is done with the simplex algorithm, and the solvers will eventually locate a global optimum if we wait long enough for the solver to finish. Otherwise, they try to find a reasonable decent solution, eventually together with a gap between the solution it identified and some hypothetical optimal solution (this is advanced material; see the videos about "Duality in Linear Programming" if you want to understand more about it). 




## Setup Your Environment/Imports

In [ ]:
# before you do anything...
# mount your drive!
# click folder on the left...
# import modules

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *

     |████████████████████████████████| 9.1 MB 15.7 MB/s 
     |████████████████████████████████| 49 kB 4.4 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155062 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to un

# Modeling Fixed Costs

A **fixed cost** is a one-time cost that must be paid to use some type of resource, regardless of the amount being used. Some examples:
* The cost to construct a **new production line**: the cost to construct a production line is not affected by the number of items you will produce in the future
* The **setup cost** required to prepare a machine for a type of product 
* The cost of **hiring additional personnel**
Fixed costs model binary decision, i.e., if you have a variable representing the decision of building a new production line, the only values that make sense for this variable are 0 and 1 (what would you do if the model returns value 0.5 for this variable?). In order to model this, you can use **binary variables**.  Binary variables are special cases of integer variables, but require variables to be either 0 or 1.

# Example: Cire Manufacturing

Cire Manufacturing can manufacture 3 types of products. Each item of these products is indivisible, i.e., you can only produce an integer amount of each type of product.

* There are 600 hours available for assembly and 300 hours available for grinding. 

* The profit per product for products 1, 2, and 3 are $\$48$, $\$55$, and $\$50$, respectively. 

* There is a production limit of 3000, 4000, and 5000 units of products 1, 2, 3, respectively.

* Setup cost to start producing for 1, 2, and 3 is $\$1000$, $\$800$, and $\$900$, respectively. 

Table with all the data is presented below:

Operation | Product 1 | Product 2 | Product 3 | Limit
--- | --- | --- | --- | ---
Assembling | 2 | 3 | 6 | 600
Grinding | 6 | 3 | 4 | 300
Profit per unit | 48 | 55 | 50 | ---
Max. Production | 3,000 | 4,000 | 5,000 | ---
Setup cost | 1000 | 800 | 900 | ---


How many products should we produce to maximize profit?


## Model overview

Our profit depends on two elements, which depend on the number of units we will produce:
* Profit per unit sold
* Setup cost
Note that it only makes sense to pay the setup cost of an item if produce at least one item of that product (otherwise, we will just waste money). Also, if we want to manufacture items of a certain product (even if it is only one unit), we need to pay the setup cost.

The setup cost for each product is a binary decision (we either manufacture items of that product or not), and we should connect this binary decision with the decision on the number of items being produced. For example, if $P_1$ is the number of items of Product $1$ and $A_1$ is a **binary variable** indicating whether we will pay the setup cost of product $1$, we should have the following:

$P_1 \leq 3,000 \cdot A_1$

The idea is actually simple:
* If $A_1 = 0$, the upper bound for $P_1$ is zero, which makes sense; if we don't pay the setup cost, we cannot produce anything
* If $A_1 = 1$, the upper bound for $P_1$ is 3,000, the maximum production level; as we paid for the setup, we can produce as much as we can of Product 1.

**Summary: Setup costs will typically be used to change bounds of (integer or continuous) variables based on the values assumed by binary activation variables.**

**Define the Objective Function**

$Max(Z) = 48P_1 + 55P_2 + 50P_3 - 1000A_1 - 800A_2 - 900A_3$ `(objective function)`

**Write the Constraints**

subject to:
* $2P_1 + 3P_2 + 6P_3 \leq 600$ `(Assembly)`
* $6P_1 + 3P_2 + 4P_3 \leq 300$ `(Grinding)`

`Production limits: upper bound is either the production limit or zero`
* $P_1        \leq 3,000A_1$ 
* $P_2        \leq 4,000A_2$ 
* $P_3        \leq 5,000A_3$ 

`Domains`
* $P_1,P_2,P_3 \in \mathbb{N}$ 
* $A_1,A_2,A_3 \in \{0,1\}$ 


In [ ]:
# declare the model
model = ConcreteModel()

# declare decision variables
model.P1 = Var(domain=Integers, bounds=(0,3000))
model.P2 = Var(domain=Integers, bounds=(0,4000))
model.P3 = Var(domain=Integers, bounds=(0,5000))

model.A1 = Var(domain=Binary)
model.A2 = Var(domain=Binary)
model.A3 = Var(domain=Binary)

# declare objective
model.profit = Objective(
                      expr = 48*model.P1 + 55*model.P2 + 50*model.P3 - 1000*model.A1 - 800*model.A2 - 900*model.A3, # values come from the table
                      sense = maximize)

# declare constraints
model.Constraint1 = Constraint(expr = 2*model.P1 + 3*model.P2 + 6*model.P3 <= 600) # assembly hours
model.Constraint2 = Constraint(expr = 6*model.P1 + 3*model.P2 + 4*model.P3 <= 300) # grinding hours
model.Constraint4 = Constraint(expr = model.P1 <= 3000*model.A1) # activation
model.Constraint5 = Constraint(expr = model.P2 <= 4000*model.A2) # activation
model.Constraint6 = Constraint(expr = model.P3 <= 5000*model.A3) # activation

# show the model you've created
model.pprint()

6 Var Declarations
    A1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    A2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    A3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    P1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  3000 : False :  True : Integers
    P2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  4000 : False :  True : Integers
    P3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  5000 : False :  True : Integers

1 Objective Declarations
    profit : Size=1, Index=None, Acti

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()
# show the results
print("Profit = ", model.profit(), " per week")
print("Product 1 = ", model.P1(), ", activation = ",model.A1())
print("Product 2 = ", model.P2(), ", activation = ",model.A2())
print("Product 3 = ", model.P3(), ", activation = ",model.A3())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4700.0
  Upper bound: 4700.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 6
  Number of binary variables: 3
  Number of integer variables: 6
  Number of nonzeros: 6
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
  